# Bathymetry

* Follow the process in Michael's
  https://github.com/SalishSeaCast/analysis-michael/blob/master/bathymetry/bathymetry-201702.ipynb
  notebook to create the base bathymetry for the 202405 coordinates and double resolution coordinates

In [2]:
import os
from pathlib import Path
import sys

import arrow
import numpy
import matplotlib.pyplot as plt
import xarray

Python and library versions:

In [3]:
import bottleneck
import h5netcdf
import h5py
import matplotlib
import netCDF4
import pandas
import scipy

print(f"Python {sys.version=}")
print(f"{numpy.__version__=}")
print(f"{xarray.__version__=}")
print(f"{bottleneck.__version__=}")
print(f"{pandas.__version__=}")
print(f"{h5netcdf.__version__=}")
print(f"{h5py.__version__=}")
print(f"{netCDF4.__version__=}")
print(f"{scipy.__version__=}")
print(f"{matplotlib.__version__=}")

Python sys.version='3.12.3 | packaged by conda-forge | (main, Apr 15 2024, 18:38:13) [GCC 12.3.0]'
numpy.__version__='1.26.4'
xarray.__version__='2024.6.0'
bottleneck.__version__='1.4.0'
pandas.__version__='2.2.2'
h5netcdf.__version__='1.3.0'
h5py.__version__='3.11.0'
netCDF4.__version__='1.6.5'
scipy.__version__='1.14.0'
matplotlib.__version__='3.8.4'


We need functions from several modules in `analysis-michael/bathymetry/` but they are not packaged.
So,
add their paths in an adjacent clone into `sys.path`.

**This is a expedient hack.
Please don't copy it unless you really know why you are doing it.**

In [4]:
# isolate the probable user-specific part of repo paths to make the notebook portable
repos_prefix = Path("/media/doug/warehouse/MEOPAR/")

In [5]:
sys.path.insert(3, os.fspath(repos_prefix / Path("analysis-michael", "bathymetry")))

sys.path

['/home/doug/.local/share/JetBrains/Toolbox/apps/pycharm-professional/plugins/python/helpers-pro/jupyter_debug',
 '/home/doug/.local/share/JetBrains/Toolbox/apps/pycharm-professional/plugins/python/helpers/pydev',
 '/media/doug/warehouse/MEOPAR/analysis-doug',
 '/media/doug/warehouse/MEOPAR/analysis-michael/bathymetry',
 '/home/doug/conda_envs/2xrez-202111/lib/python312.zip',
 '/home/doug/conda_envs/2xrez-202111/lib/python3.12',
 '/home/doug/conda_envs/2xrez-202111/lib/python3.12/lib-dynload',
 '',
 '/home/doug/conda_envs/2xrez-202111/lib/python3.12/site-packages']

In [6]:
from bathy_readers import getnemo